In [ ]:
import pandas as pd
import numpy as np

import requests
from bs4 import BeautifulSoup
import json
from urllib.request import urlopen

from tqdm import tqdm_notebook
import unidecode

In [ ]:
df_debut = pd.read_csv("../data/debut_album_1118-utf8.csv")
df_debut.head()

In [ ]:
def get_xxl_articles():
    
    df_articles = pd.DataFrame(columns = ['artist', 'album', 'xxl_frequency'])
    
    for i in tqdm_notebook(range(len(list(df_debut['artist'])))):
        
        artist = df_debut['artist'][i]
        album = df_debut['album'][i]

        artist_ = artist.replace(" ", "%20")
        album_ = album.replace(" ", "%20")

        keyword = artist_ + "%20" + album_

        url = 'http://www.xxlmag.com/rest/carbon/api/searchfor/widgetsearch/?s=' + keyword + '&searchpage=1'
        response = requests.get(url).json()

        articles = response['data']['carbonwidget/searchnative-1']['params']['total']

        data = {
                'artist': artist,
                'album' : album,
                'xxl_frequency' : articles
            }

#         print(data)

        df_articles.loc[len(df_articles)] = data
    
    return df_articles

In [ ]:
xxl_articles = get_xxl_articles()

In [ ]:
print(xxl_articles.shape)
xxl_articles.head()

In [ ]:
def decodeuni(str):
    return unidecode.unidecode(str)

In [ ]:
xxl_articles['artist'] = xxl_articles['artist'].apply(lambda x: unidecode.unidecode(x))
xxl_articles['album'] = xxl_articles['album'].apply(lambda x: unidecode.unidecode(x))
xxl_articles['artist'] = xxl_articles['artist'].apply(lambda x : x.replace(",", ""))
xxl_articles['album'] = xxl_articles['album'].apply(lambda x : x.replace(",", ""))

In [ ]:
xxl_articles.loc[2, 'album'] = '9am to 5pm 5pm to Whenever'
xxl_articles.loc[529, 'album'] = 'If You Love Me'
xxl_articles.loc[1143, 'album'] = 'Gover Tela Body Language'
xxl_articles.loc[1411, 'album'] = 'Fresh Filter Volumn 1'

In [ ]:
xxl_articles.to_csv("../data/buzz_xxl.csv", index=False, encoding='UTF-8')

In [ ]:
df = pd.read_csv("../data/buzz/buzz_xxl.csv", encoding="UTF-8")

In [ ]:
print(df.shape)
df.head()

In [ ]:
df.info()

### Upload to MySQL database

In [ ]:
import sqlalchemy, pickle
from sqlalchemy import create_engine

In [ ]:
pw = pickle.load(open("mysql_pw.pickle", "rb"))

In [ ]:
engine = sqlalchemy.create_engine("mysql+mysqldb://root:" + pw + "@52.78.44.120/project_rookie")

In [ ]:
df.to_sql(name="buzz_xxl", con=engine, if_exists='replace')